# Schwarz IT: Einfache Anomalie-Erkennung in Server-Logs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_10/schwarz_it_simple.ipynb)

**Lernziel:** Grundlagen der Anomalie-Erkennung - Erkennung ungewöhnlicher Muster in IT-Systemen

**Vereinfachtes Szenario:** Überwachung der Server-Response-Times und Erkennung von Performance-Problemen

**Lehrbeispiel!** Die Daten sind künstlich erzeugt und ohne echte Zusammenhänge! Es gibt keinen Bezug zu Schwarz IT. :-)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Simuliere Server-Response-Times über 24 Stunden
np.random.seed(42)
n_minutes = 24 * 60  # 24 Stunden in Minuten

# Zeitstempel erstellen
start_time = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
timestamps = [start_time + timedelta(minutes=i) for i in range(n_minutes)]

# Normale Response Times (normalverteilt um 200ms)
normal_response = np.random.normal(200, 30, n_minutes)

# Einige Anomalien hinzufügen (Server-Probleme)
anomaly_indices = [300, 301, 302, 800, 801, 1200, 1201, 1202, 1203]  # Problemphasen
response_times = normal_response.copy()
response_times[anomaly_indices] = np.random.normal(800, 100, len(anomaly_indices))  # Deutlich langsamer

df = pd.DataFrame({
    'timestamp': timestamps,
    'response_time_ms': response_times.round(1)
})

df['hour'] = df['timestamp'].dt.hour
print(f"Server-Logs für {n_minutes/60:.0f} Stunden")
print("\nErste 5 Einträge:")
df.head()

In [ ]:
# Histogramm der Response Times - Baseline für Anomalie-Erkennung
plt.figure(figsize=(12, 6))

# Histogramm aller Response Times
plt.hist(df['response_time_ms'], bins=50, alpha=0.7, color='lightblue', 
         edgecolor='black', label='Server Response Times')

# Basis-Statistiken
mean_response = df['response_time_ms'].mean()
std_response = df['response_time_ms'].std()

# Statistische Marker
plt.axvline(mean_response, color='green', linestyle='-', linewidth=2, 
           label=f'Mittelwert ({mean_response:.0f} ms)')
plt.axvline(mean_response + 3*std_response, color='red', linestyle='--', linewidth=2, 
           label=f'3σ-Grenze ({mean_response + 3*std_response:.0f} ms)')

plt.xlabel('Response Time (ms)')
plt.ylabel('Häufigkeit (Anzahl Requests)')
plt.title('Schwarz IT: Verteilung der Server Response Times\nGrundlage für einfache Anomalie-Erkennung')
plt.legend()
plt.grid(True, alpha=0.3)

# Statistik-Box
stats_text = f'Normalverteilung:\nμ = {mean_response:.1f} ms\nσ = {std_response:.1f} ms\n3σ-Regel: 99.7% < {mean_response + 3*std_response:.0f} ms'
plt.text(0.98, 0.98, stats_text, transform=plt.gca().transAxes, 
         verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
         fontsize=10)

plt.tight_layout()
plt.show()

print("📊 AUSGANGSLAGE:")
print(f"• Die Response Times folgen einer Normalverteilung um {mean_response:.0f} ms")
print(f"• 99.7% aller Werte liegen unter {mean_response + 3*std_response:.0f} ms (3σ-Regel)")
print(f"• Werte darüber sind statistisch 'ungewöhnlich' → potentielle Anomalien")

In [ ]:
# Basis-Statistiken berechnen
mean_response = df['response_time_ms'].mean()
std_response = df['response_time_ms'].std()
threshold = mean_response + 3 * std_response  # 3-Sigma-Regel

print("=== SCHWARZ IT - SERVER MONITORING ===")
print(f"Durchschnittliche Response Time: {mean_response:.1f} ms")
print(f"Standardabweichung: {std_response:.1f} ms")
print(f"Anomalie-Schwellwert (μ + 3σ): {threshold:.1f} ms")

# Anomalien identifizieren
df['is_anomaly'] = df['response_time_ms'] > threshold
n_anomalies = df['is_anomaly'].sum()

print(f"\n🚨 Anzahl Anomalien gefunden: {n_anomalies}")
print(f"Anomalie-Rate: {n_anomalies/len(df)*100:.2f}%")

In [ ]:
# Visualisierung der Response Times und Anomalien
plt.figure(figsize=(15, 8))

# Normale Zeiten in blau
normal_data = df[~df['is_anomaly']]
plt.scatter(normal_data['hour'], normal_data['response_time_ms'], 
           alpha=0.6, color='blue', s=20, label='Normal')

# Anomalien in rot
anomaly_data = df[df['is_anomaly']]
plt.scatter(anomaly_data['hour'], anomaly_data['response_time_ms'], 
           alpha=0.8, color='red', s=50, label='Anomalie', marker='x')

# Schwellwert als horizontale Linie
plt.axhline(y=threshold, color='orange', linestyle='--', linewidth=2, 
           label=f'Schwellwert ({threshold:.0f} ms)')

plt.xlabel('Stunde des Tages')
plt.ylabel('Response Time (ms)')
plt.title('Schwarz IT: Server Response Time Monitoring')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Detaillierte Anomalie-Analyse
if n_anomalies > 0:
    print("=== ANOMALIE-DETAILS ===")
    anomaly_details = df[df['is_anomaly']][['timestamp', 'response_time_ms']].copy()
    anomaly_details['zeit'] = anomaly_details['timestamp'].dt.strftime('%H:%M')
    
    for _, row in anomaly_details.iterrows():
        print(f"⚠️  {row['zeit']} Uhr: {row['response_time_ms']:.1f} ms (Normal: ~{mean_response:.0f} ms)")
    
    print("\n=== AUTOMATISCHE ALERTS ===")
    worst_response = df['response_time_ms'].max()
    worst_time = df[df['response_time_ms'] == worst_response]['timestamp'].iloc[0]
    
    print(f"🔴 KRITISCH: Langsamste Response um {worst_time.strftime('%H:%M')} Uhr: {worst_response:.1f} ms")
    print(f"📱 SMS an On-Call Engineer gesendet")
    print(f"📧 E-Mail an IT-Team gesendet")
else:
    print("✅ Keine Anomalien gefunden - Server läuft stabil!")

## IT Operations Impact

**Business Value:**
- 🚨 **Frühwarnsystem:** Probleme erkennen bevor Kunden betroffen sind
- ⚡ **Schnelle Reaktion:** Automatische Alerts für kritische Situationen  
- 📊 **Performance Trends:** Langfristige Überwachung der System-Gesundheit
- 💰 **Kosteneinsparung:** Weniger Downtimes und bessere Customer Experience

**Einfache Regel:** Response Times > 3 Standardabweichungen = Anomalie

**Nächste Schritte:** Machine Learning für komplexere Muster, Integration mit Monitoring-Tools